In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Utils

### Opts

In [8]:
import tempfile

masif_opts = {}
# Default directories
masif_opts["raw_pdb_dir"] = "../data/data_preparation/raw_pdbs"
masif_opts["pdb_chain_dir"] = "../data/data_preparation/01-benchmark_pdbs/"
masif_opts["ply_chain_dir"] = "../data/data_preparation/01-benchmark_surfaces/"
masif_opts["tmp_dir"] = tempfile.gettempdir()
masif_opts["ply_file_template"] = masif_opts["ply_chain_dir"] + "/{}_{}.ply"

# Surface features
masif_opts["use_hbond"] = True
masif_opts["use_hphob"] = True
masif_opts["use_apbs"] = True
masif_opts["compute_iface"] = True
# Mesh resolution. Everything gets very slow if it is lower than 1.0
masif_opts["mesh_res"] = 1.0
masif_opts["feature_interpolation"] = True


# Coords params
masif_opts["radius"] = 12.0

# Neural network patch application specific parameters.
masif_opts["ppi_search"] = {}
masif_opts["ppi_search"]["training_list"] = "../data/lists/training.txt"
masif_opts["ppi_search"]["testing_list"] = "../data/lists/testing.txt"
masif_opts["ppi_search"]["max_shape_size"] = 200
masif_opts["ppi_search"]["max_distance"] = 12.0  # Radius for the neural network.
masif_opts["ppi_search"][
    "masif_precomputation_dir"
] = "../data/data_preparation/04b-precomputation_12A/precomputation/"
masif_opts["ppi_search"]["feat_mask"] = [1.0] * 5
masif_opts["ppi_search"]["max_sc_filt"] = 1.0
masif_opts["ppi_search"]["min_sc_filt"] = 0.5
masif_opts["ppi_search"]["pos_surf_accept_probability"] = 1.0
masif_opts["ppi_search"]["pos_interface_cutoff"] = 1.0
masif_opts["ppi_search"]["range_val_samples"] = 0.9  # 0.9 to 1.0
masif_opts["ppi_search"]["cache_dir"] = "nn_models/sc05/cache/"
masif_opts["ppi_search"]["model_dir"] = "nn_models/sc05/all_feat/model_data/"
masif_opts["ppi_search"]["desc_dir"] = "descriptors/sc05/all_feat/"
masif_opts["ppi_search"]["gif_descriptors_out"] = "gif_descriptors/"
# Parameters for shape complementarity calculations.
masif_opts["ppi_search"]["sc_radius"] = 12.0
masif_opts["ppi_search"]["sc_interaction_cutoff"] = 1.5
masif_opts["ppi_search"]["sc_w"] = 0.25

# Neural network patch application specific parameters.
masif_opts["site"] = {}
masif_opts["site"]["training_list"] = "lists/training.txt"
masif_opts["site"]["testing_list"] = "lists/testing.txt"
masif_opts["site"]["max_shape_size"] = 100
masif_opts["site"]["n_conv_layers"] = 3
masif_opts["site"]["max_distance"] = 9.0  # Radius for the neural network.
masif_opts["site"][
    "masif_precomputation_dir"
] = "data_preparation/04a-precomputation_9A/precomputation/"
masif_opts["site"]["range_val_samples"] = 0.9  # 0.9 to 1.0
masif_opts["site"]["model_dir"] = "nn_models/all_feat_3l/model_data/"
masif_opts["site"]["out_pred_dir"] = "output/all_feat_3l/pred_data/"
masif_opts["site"]["out_surf_dir"] = "output/all_feat_3l/pred_surfaces/"
masif_opts["site"]["feat_mask"] = [1.0] * 5

# Neural network ligand application specific parameters.
masif_opts["ligand"] = {}
masif_opts["ligand"]["assembly_dir"] = "data_preparation/00b-pdbs_assembly"
masif_opts["ligand"]["ligand_coords_dir"] = "data_preparation/00c-ligand_coords"
masif_opts["ligand"][
    "masif_precomputation_dir"
] = "data_preparation/04a-precomputation_12A/precomputation/"
masif_opts["ligand"]["max_shape_size"] = 200
masif_opts["ligand"]["feat_mask"] = [1.0] * 5
masif_opts["ligand"]["train_fract"] = 0.9 * 0.8
masif_opts["ligand"]["val_fract"] = 0.1 * 0.8
masif_opts["ligand"]["test_fract"] = 0.2
masif_opts["ligand"]["tfrecords_dir"] = "data_preparation/tfrecords"
masif_opts["ligand"]["max_distance"] = 12.0
masif_opts["ligand"]["n_classes"] = 7
masif_opts["ligand"]["feat_mask"] = [1.0, 1.0, 1.0, 1.0, 1.0]
masif_opts["ligand"]["costfun"] = "dprime"
masif_opts["ligand"]["model_dir"] = "nn_models/all_feat/"
masif_opts["ligand"]["test_set_out_dir"] = "test_set_predictions/"

## Data Preparation

### PDB Download

In [ ]:
import os
import pandas as pd
from Bio.PDB import PDBList

def pdb_download(tsv_file_path, pdb_directory):
    """
    Lädt PDB-Dateien für die in der TSV-Datei angegebenen PDB-IDs herunter.

    :param tsv_file_path: Pfad zur TSV-Datei mit den PDB-IDs
    :param pdb_directory: Verzeichnis, in dem die PDB-Dateien gespeichert werden sollen
    """

    # Lese die TSV-Datei und extrahiere die PDB-IDs
    data = pd.read_csv(tsv_file_path, sep='\t')
    # Nimm nur die ersten 10 Einträge (zum Testen)
    data = data.head(10)
    # Nimm nur eindeute PDB-IDs und entferne alle unnötigen Zeichen
    data['PDB'] = data['PDB'].apply(lambda x: x.split(';')[0].strip().split(',')[0].strip())
    pdb_ids = data['PDB'].dropna().unique()

    # Erstellen eines Verzeichnisses für die heruntergeladenen PDB-Dateien
    pdb_directory = '../data/pdb_files'
    os.makedirs(pdb_directory, exist_ok=True)

    # PDB-Downloader initialisieren
    pdbl = PDBList()

    # Durchlaufe alle PDB-IDs und lade die Dateien herunter
    for pdb_id in pdb_ids:
        print(f"Downloading PDB file for {pdb_id}...")
        try:
            # Versuche die PDB-Datei herunterzuladen
            filename = pdbl.retrieve_pdb_file(pdb_id, pdir=pdb_directory, file_format='pdb')
            correct_filename = os.path.join(pdb_directory, f"{pdb_id.lower()}.pdb")
            if not filename.endswith('.pdb'):
                os.rename(filename, correct_filename)
            print(f"PDB file saved as: {correct_filename}")
        except Exception as e:
            print(f"Error processing {pdb_id}: {e}")

    print("All PDB files downloaded.")
    print(f"Saved in directory: {pdb_directory}")

    return pdb_directory

pdb_download("../data/data_preparation/uniprot/raw_enzyme_list.tsv", "../data/data_preparation/raw_pdbs")

### Generate Assembly

In [ ]:
import os
import sys
from SBILib.structure import PDB


def generate_assembly(pdb_id, raw_pdb_dir, assembly_dir):
    """
    Lädt eine PDB-Datei herunter und generiert die biologische Assemblierung.

    :param pdb_id: PDB-ID
    :param raw_pdb_dir: Verzeichnis, in dem die PDB-Dateien gespeichert werden sollen
    :param assembly_dir: Verzeichnis, in dem die biologischen Assemblierungen gespeichert werden sollen
    """
    

    ligands = ["ADP", "COA", "FAD", "HEM", "NAD", "NAP", "SAM"]

    if not os.path.exists(assembly_dir):
        os.mkdir(assembly_dir)

    print(pdb_id)

    def assemble(pdb_id):
        # Reads and builds the biological assembly of a structure
        print("Building assembly for", pdb_id)
        struct = PDB(os.path.join(raw_pdb_dir, pdb_id + ".pdb"))
        try:
            struct_assembly = struct.apply_biomolecule_matrices()[0]
        except:
            return 0
        struct_assembly.write(
            os.path.join(assembly_dir, "{}.pdb".format(pdb_id))
        )
        return 1


    in_fields = sys.argv[1].split("_")
    #pdb_id = in_fields[0]

    res = assemble(pdb_id)
    if res:
        print("Building assembly was successfull for {}".format(pdb_id))
    else:
        print("Building assembly was not successfull for {}".format(pdb_id))

# Generate assemblies for all PDB files
raw_pdb_dir = "../data/data_preparation/raw_pdbs"
assembly_dir = "../data/data_preparation/assembly_dir"
pdb_files = os.listdir(raw_pdb_dir)
pdb_ids = [f.split(".")[0] for f in pdb_files]
for pdb_id in pdb_ids:
    generate_assembly(pdb_id, raw_pdb_dir, assembly_dir)

### Save Ligand Coords

In [ ]:
import os
import numpy as np
import sys

from SBILib.structure import PDB

def save_ligand_coords(pdb_id, ligand_coords_dir, assembly_dir):

    if not os.path.exists(ligand_coords_dir):
        os.mkdir(ligand_coords_dir)

    # Ligands of interest
    ligands = ["ADP", "COA", "FAD", "HEM", "NAD", "NAP", "SAM"]

    structure_ligands_type = []
    structure_ligands_coords = []

    try:
        structure = PDB(os.path.join(assembly_dir, pdb_id + ".pdb"))
    except:
        print("Problem with opening structure", pdb_id)
    for chain in structure.chains:
        for het in chain.heteroatoms:
            # Check all ligands in structure and save coordinates if they are of interest
            if het.type in ligands:
                structure_ligands_type.append(het.type)
                structure_ligands_coords.append(het.all_coordinates)

    np.save(
        os.path.join(
            ligand_coords_dir, "{}_ligand_types.npy".format(pdb_id)
        ),
        structure_ligands_type,
    )
    np.save(
        os.path.join(
            ligand_coords_dir, "{}_ligand_coords.npy".format(pdb_id)
        ),
        structure_ligands_coords,
    )

# Save ligand coordinates for all PDB files
ligand_coords_dir = "../data/data_preparation/ligand_coords"
assembly_dir = "../data/data_preparation/assembly_dir"
pdb_files = os.listdir(assembly_dir)
pdb_ids = [f.split(".")[0] for f in pdb_files]
for pdb_id in pdb_ids:
    save_ligand_coords(pdb_id, ligand_coords_dir, assembly_dir)

### Extract and Triangulate

In [23]:
#!/usr/bin/python
import numpy as np
import os
import Bio
import shutil
from Bio.PDB import * 
import sys
import importlib
from IPython.core.debugger import set_trace

#import pymesh

# Local includes
import sys  
sys.path.insert(1, '../masif_seed/masif/source/triangulation')
sys.path.insert(1, '../masif_seed/masif/source/input_output')
import computeMSMS
import fixmesh
import extractPDB
import save_ply
import read_ply
import protonate
import computeHydrophobicity
import computeCharges, assignChargesToNewMesh
import computeAPBS
import compute_normal
from sklearn.neighbors import KDTree

def extract_and_triangulate(masif_opts, pdb_id):

    # Save the chains as separate files.
    in_fields = sys.argv[1].split("_")
    pdb_id = in_fields[0]
    chain_ids1 = in_fields[1]

    if (len(sys.argv)>2) and (sys.argv[2]=='masif_ligand'):
        pdb_filename = os.path.join(masif_opts["ligand"]["assembly_dir"],pdb_id+".pdb")
    else:
        pdb_filename = masif_opts['raw_pdb_dir']+pdb_id+".pdb"
    tmp_dir= masif_opts['tmp_dir']
    protonated_file = tmp_dir+"/"+pdb_id+".pdb"
    protonate(pdb_filename, protonated_file)
    pdb_filename = protonated_file

    # Extract chains of interest.
    out_filename1 = tmp_dir+"/"+pdb_id+"_"+chain_ids1
    extractPDB(pdb_filename, out_filename1+".pdb", chain_ids1)

    # Compute MSMS of surface w/hydrogens, 
    try:
        vertices1, faces1, normals1, names1, areas1 = computeMSMS(out_filename1+".pdb",\
            protonate=True)
    except:
        set_trace()

    # Compute "charged" vertices
    if masif_opts['use_hbond']:
        vertex_hbond = computeCharges(out_filename1, vertices1, names1)

    # For each surface residue, assign the hydrophobicity of its amino acid. 
    if masif_opts['use_hphob']:
        vertex_hphobicity = computeHydrophobicity(names1)

    # If protonate = false, recompute MSMS of surface, but without hydrogens (set radius of hydrogens to 0).
    vertices2 = vertices1
    faces2 = faces1

    # Fix the mesh.
    mesh = pymesh.form_mesh(vertices2, faces2)
    regular_mesh = fix_mesh(mesh, masif_opts['mesh_res'])

    # Compute the normals
    vertex_normal = compute_normal(regular_mesh.vertices, regular_mesh.faces)
    # Assign charges on new vertices based on charges of old vertices (nearest
    # neighbor)

    if masif_opts['use_hbond']:
        vertex_hbond = assignChargesToNewMesh(regular_mesh.vertices, vertices1,\
            vertex_hbond, masif_opts)

    if masif_opts['use_hphob']:
        vertex_hphobicity = assignChargesToNewMesh(regular_mesh.vertices, vertices1,\
            vertex_hphobicity, masif_opts)

    if masif_opts['use_apbs']:
        vertex_charges = computeAPBS(regular_mesh.vertices, out_filename1+".pdb", out_filename1)

    iface = np.zeros(len(regular_mesh.vertices))
    if 'compute_iface' in masif_opts and masif_opts['compute_iface']:
        # Compute the surface of the entire complex and from that compute the interface.
        v3, f3, _, _, _ = computeMSMS(pdb_filename,\
            protonate=True)
        # Regularize the mesh
        mesh = pymesh.form_mesh(v3, f3)
        # I believe It is not necessary to regularize the full mesh. This can speed up things by a lot.
        full_regular_mesh = mesh
        # Find the vertices that are in the iface.
        v3 = full_regular_mesh.vertices
        # Find the distance between every vertex in regular_mesh.vertices and those in the full complex.
        kdt = KDTree(v3)
        d, r = kdt.query(regular_mesh.vertices)
        d = np.square(d) # Square d, because this is how it was in the pyflann version.
        assert(len(d) == len(regular_mesh.vertices))
        iface_v = np.where(d >= 2.0)[0]
        iface[iface_v] = 1.0
        # Convert to ply and save.
        save_ply(out_filename1+".ply", regular_mesh.vertices,\
                            regular_mesh.faces, normals=vertex_normal, charges=vertex_charges,\
                            normalize_charges=True, hbond=vertex_hbond, hphob=vertex_hphobicity,\
                            iface=iface)

    else:
        # Convert to ply and save.
        save_ply(out_filename1+".ply", regular_mesh.vertices,\
                            regular_mesh.faces, normals=vertex_normal, charges=vertex_charges,\
                            normalize_charges=True, hbond=vertex_hbond, hphob=vertex_hphobicity)
    if not os.path.exists(masif_opts['ply_chain_dir']):
        os.makedirs(masif_opts['ply_chain_dir'])
    if not os.path.exists(masif_opts['pdb_chain_dir']):
        os.makedirs(masif_opts['pdb_chain_dir'])
    shutil.copy(out_filename1+'.ply', masif_opts['ply_chain_dir']) 
    shutil.copy(out_filename1+'.pdb', masif_opts['pdb_chain_dir'])

# Extract and triangulate all PDB files
pdb_files = os.listdir(masif_opts['raw_pdb_dir'])
pdb_ids = [f.split(".")[0] for f in pdb_files]
for pdb_id in pdb_ids:
    extract_and_triangulate(masif_opts, pdb_id)

> /Users/tobias.polley/Repositories/DeepZyme/masif_seed/masif/source/default_config/global_vars.py(14)<module>()
     12 else:
     13   set_trace()
---> 14   print("ERROR: MSMS_BIN not set. Variable should point to MSMS program.")
     15   sys.exit(1)
     16 



In [ ]:
import os
from Bio.PDB import PDBParser, Select

class NonHeteroSelect(Select):
    """ Klasse, um heterogene Atome aus der Struktur zu entfernen. """
    def accept_residue(self, residue):
        return residue.id[0] == ' '

# Verzeichnis mit den PDB-Dateien
pdb_directory = '../data/pdb_files'
processed_directory = '../data/processed_pdb_files'
os.makedirs(processed_directory, exist_ok=True)

# PDB-Parser initialisieren
parser = PDBParser()

# Liste der PDB-Dateien
pdb_files = [f for f in os.listdir(pdb_directory) if f.endswith('.pdb')]
print(f"Found {len(pdb_files)} PDB files in {pdb_directory}")

# Durchlaufe alle PDB-Dateien
for pdb_file in pdb_files:
    structure_id = pdb_file.split('.')[0]
    filename = os.path.join(pdb_directory, pdb_file)
    structure = parser.get_structure(structure_id, filename)

    # Entferne heterogene Atome
    from Bio.PDB import PDBIO
    io = PDBIO()
    io.set_structure(structure)
    processed_filename = os.path.join(processed_directory, pdb_file)
    io.save(processed_filename, NonHeteroSelect())

    print(f"Processed {pdb_file} and saved to {processed_filename}")